## Домашнее задание по блоку 19.04.2021. Кейс-стади 
### Тихонова Инна

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

**Задание 1**

Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1. Как критики относятся к спортивным играм?
2. Критикам нравятся больше игры на PC или на PS4?
3. Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:
 - сформулируйте нулевую и альтернативную гипотезы;
 - выберите пороговый уровень статистической значимости;
 - опишите полученные результаты статистического теста.

In [2]:
df = pd.read_csv('video_games_sales.csv')
df.columns = df.columns.str.lower()
df.head()

,rank,name,basename,genre,esrb_rating,platform,publisher,developer,vgchartz_score,critic_score,...,na_sales,pal_sales,jp_sales,other_sales,year,last_update,url,status,vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


**1. Как критики относятся к спортивным играм?**

Критики ставят играм в основом высокие оценки (выше 7)?

 - H0: средняя оценка игр критиками <= 7.0.
 - H1: средняя оценка игр критиками > 7.0.

За alpha здесь и далее будем принимать 0.05

In [3]:
df_cs = df[np.logical_not(df['critic_score'].isna())]['critic_score']

In [4]:
alpha = 0.05
result = st.ttest_1samp(df_cs, 7.0, alternative='greater')

print(df_cs.mean())
print(result)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 7.0')
else:
    print('Не отвергаем нулевую нулевую гипотезу, средняя оценка игр критиками не выше 7.0')

7.21370869033052
Ttest_1sampResult(statistic=11.882016908955192, pvalue=1.5730299883407045e-32)
Отвергаем нулевую гипотезу, средняя оценка игр критиками выше 7.0


**2. Критикам нравятся больше игры на PC или на PS4?**

Средняя оценка игр на PC и на PS4 одинакова?

 - H0: средняя оценка игр на PC и на PS4 одинакова.
 - H1: средняя оценка игр на PC и на PS4 различается.

In [5]:
df_cs_PC = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['platform'] == 'PC')]['critic_score']

df_cs_PS4 = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['platform'] == 'PS4')]['critic_score']

In [6]:
result = st.ttest_ind(df_cs_PC, df_cs_PS4, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 одинакова')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, средняя оценка игр на PC и на PS4 различается


**3. Критикам больше нравятся стрелялки или стратегии?**

Средняя оценка шутеров и стратегий одинакова?

 - H0: средняя оценка шутеров и стратегий одинакова.
 - H1: средняя оценка шутеров и стратегий различается.

In [7]:
df_cs_sh = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['genre'] == 'Shooter')]['critic_score']

df_cs_st = df[np.logical_and(np.logical_not(df['critic_score'].isna()),
                                    df['genre'] == 'Strategy')]['critic_score']

In [8]:
result = st.ttest_ind(df_cs_sh, df_cs_st, equal_var=False)
print(result)

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий различается')
else:
    print('Не отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий одинакова')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу, средняя оценка шутеров и стратегий одинакова


**Задание 2**

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1. Приведите весь текст к нижнему регистру;
2. Удалите мусорные символы;
3. Удалите стоп-слова;
4. Приведите все слова к нормальной форме;
5. Преобразуйте все сообщения в вектора TF-IDF; 
6. Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;
7. Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
8. Опишите результаты при помощи confusion_matrix;
9. Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [9]:
df = pd.read_csv('spam.csv')
df.columns = df.columns.str.lower()
df.head()

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**1. Приведите весь текст к нижнему регистру**

In [10]:
df['message'] = df['message'].str.lower()
df.head()

,category,message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


**2. Удалите мусорные символы**

In [11]:
df['only_words'] = df['message'].map(lambda x: re.sub('[\W_]+',' ', x))
df.head()

,category,message,only_words
0,ham,"go until jurong point, crazy.. available only ...",go until jurong point crazy available only in ...
1,ham,ok lar... joking wif u oni...,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...,u dun say so early hor u c already then say
4,ham,"nah i don't think he goes to usf, he lives aro...",nah i don t think he goes to usf he lives arou...


**3. Удалите стоп-слова**

In [12]:
df['only_words'] = df['only_words'].map(lambda x: x.split())
df.head()

,category,message,only_words
0,ham,"go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o..."
1,ham,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,..."


In [13]:
stopwords_set = set(stopwords.words('english'))
df['without_sw'] = df['only_words'].map(lambda x: [word for word in x if word not in stopwords_set] )
df.head()

,category,message,only_words,without_sw
0,ham,"go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]"


**4. Приведите все слова к нормальной форме**

In [14]:
wordnet_lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['without_sw'].map(lambda x: [wordnet_lemmatizer.lemmatize(word) for word in x] )
df.head()

,category,message,only_words,without_sw,lemmatized
0,ham,"go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, go, usf, life, around, though]"


In [15]:
def to_str(lem):
    return ' '.join([x for x in lem])

In [16]:
df['new_message'] = df['lemmatized'].apply(to_str)
df.head()

,category,message,only_words,without_sw,lemmatized,new_message
0,ham,"go until jurong point, crazy.. available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n...",go jurong point crazy available bugis n great ...
1,ham,ok lar... joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin...",free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say so early hor... u c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]",u dun say early hor u c already say
4,ham,"nah i don't think he goes to usf, he lives aro...","[nah, i, don, t, think, he, goes, to, usf, he,...","[nah, think, goes, usf, lives, around, though]","[nah, think, go, usf, life, around, though]",nah think go usf life around though


**5. Преобразуйте все сообщения в вектора TF-IDF**

In [17]:
df = df.drop(columns=['message', 'only_words', 'without_sw', 'lemmatized'])
df.head()

,category,new_message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though


In [18]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['new_message'])

names = tfidf.get_feature_names()
matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
matrix.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**6. Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42.**

In [19]:
df['class_message'] = (df['category'] == 'spam') * 1
df.head()

,category,new_message,class_message
0,ham,go jurong point crazy available bugis n great ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,u dun say early hor u c already say,0
4,ham,nah think go usf life around though,0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(matrix, df['class_message'], test_size=0.30, random_state=42)

**7. Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных**

In [21]:
lda = LinearDiscriminantAnalysis()

In [22]:
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [23]:
accuracy_score(y_test, lda.predict(X_test))

0.9677033492822966

Точность - 96%.

**8. Опишите результаты при помощи confusion_matrix**

In [24]:
confusion_matrix(y_test, lda.predict(X_test))

array([[1445,    3],
       [  51,  173]], dtype=int64)

Правильно предсказано 1445 единичек и 173 ноля. Также есть 51 + 3 ошибки (ноль вместо единицы или наоборот).

**9. Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного)**

In [25]:
df_bad = pd.concat([df[df.index.isin(y_test.index)].reset_index(), 
                    pd.Series(lda.predict(X_test),
                              np.arange(len(lda.predict(X_test))),
                              name = 'predict')],
                    axis=1)
df_bad = df_bad[df_bad['class_message'] != df_bad['predict']]
df_bad

,index,category,new_message,class_message,predict
0,8,spam,winner valued network customer selected receiv...,1,0
1,12,spam,urgent 1 week free membership 100 000 prize ja...,1,0
2,15,spam,xxxmobilemovieclub use credit click wap link n...,1,0
4,19,spam,england v macedonia dont miss goal team news t...,1,0
14,47,ham,fair enough anything going,0,1
...,...,...,...,...,...
1633,5446,ham,back good journey let know need receipt shall ...,0,1
1634,5450,ham,sac need carry,0,1
1638,5457,ham,arun u transfr amt,0,1
1657,5524,spam,awarded sipix digital camera call 09061221061 ...,1,0
